In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gsd.hoomd
import numpy.linalg as la
import pandas as pd

import warnings
import freud
import freud.locality
from typing import Optional
import scipy.ndimage

from numba import njit

In [ ]:
from schmeud import softness
import pickle

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
traj = gsd.hoomd.open("../test_fire.gsd")

In [ ]:
snap = traj[0]

In [ ]:
@njit
def cube_pbc_wrap(init_x, final_x, box):
    out = np.zeros(3)
    for i in range(3):
        b = box[i]
        v = init_x[i]
        w = final_x[i]
        b2 = .5*b
        dx = w - v
        if dx > b2:
            dx -= b
        elif dx < -b2:
            dx += b
        out[i] = dx
    return out

box = snap.configuration.box

cube_pbc_wrap(np.array([0.0, 0.0, 0.0]), np.array([3.0, 2.5, 4.0]), box)

In [ ]:


def p_hop_interal(
        pos: np.ndarray,
        tr_frames: int
) -> np.ndarray:
    """Fast implementation of phop using numba.

    Scans through the array of postions and calculates phop with the given t_r.

    Arguments
    ---------
    * pos: 3D ndarray of particle coordinates.
    * tr_frames: Size of the scanning window, t_r. Actual calculation will use
        one additional frame at the beginning of the interval.

    Returns
    -------
    * p_hop: 2D ndarray quantifying dynamical activity.
    """

    n_frames = len(pos)
    half = int(tr_frames/2)

    phop = np.zeros((n_frames - tr_frames, len(pos[0])))

    for i in range(len(phop)):
        r_A = pos[i:i+half+1]
        r_B = pos[i+half:i+tr_frames+1]

        # phop[i] = p_hop_calc(r_A, r_B)

        phop[i] = np.sqrt(
            np.mean(np.sum(np.square(r_A - np.mean(r_B, axis=0)), axis=-1), axis=0) *
            np.mean(np.sum(np.square(r_B - np.mean(r_A, axis=0)), axis=-1), axis=0)
        )

    return phop

def get_freud_box(snapshot: gsd.hoomd.Snapshot) -> freud.box.Box:

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        box: freud.box.Box = freud.box.Box(
            *snapshot.configuration.box,
            snapshot.configuration.dimensions == 2)
        return box

def calc_phop(
        traj: gsd.hoomd.HOOMDTrajectory,
        tr_frames: int = 10,
        time_ave_frames: Optional[int] = None
) -> np.ndarray:

    assert(tr_frames % 2 == 0)
    assert(tr_frames > 0)
    assert(len(traj) > tr_frames)

    n_frames = len(traj)
    init_pos = traj[0].particles.position
    pos = np.zeros((n_frames, *init_pos.shape))
    pos[0] = init_pos
    unwrap = np.zeros(init_pos.shape, dtype=np.int64)
    box = get_freud_box(traj[0])
    L = box.Lx
    L2 = L/2.0

    # wrap box positions, should probably refactor this
    # this assumes that the box does not change during the simulation
    for i in range(1, n_frames):
        pos[i] = traj[i].particles.position
        diff = pos[i] - init_pos
        unwrap += np.trunc(diff/L2).astype(np.int64)
        init_pos = pos[i].copy()
        pos[i] -= unwrap.astype(np.float64)*L

    if time_ave_frames is not None:
        pos = scipy.ndimage.uniform_filter1d(pos, time_ave_frames, axis=0)

    phop = p_hop_interal(pos, tr_frames)  # calc p_hop

    return phop

In [ ]:
phop = calc_phop(traj, tr_frames=100)

In [ ]:
test_id = 200

In [ ]:
plt.hist(phop[:,test_id], bins=30)
plt.yscale('log')
plt.xlabel(r'$p_{hop}$')
plt.ylabel(r"$N$")

In [ ]:
plt.plot(phop[:,test_id])
plt.ylabel(r'$p_{hop}$')
plt.xlabel(r"$t$")

In [ ]:
plt.plot(phop[:,test_id])
plt.ylabel(r'$p_{hop}$')
plt.xlabel(r"$t$")

In [ ]:
test_id = 200
start = traj[0].particles.position[test_id]
box = traj[0].configuration.box # this box is constant throughout the sim
out = []
for snap in traj:
    next_pos = snap.particles.position[test_id]
    wrapped_vec = cube_pbc_wrap(start, next_pos, box)
    norm = la.norm(wrapped_vec)
    out.append(norm)
plt.hist(out, bins=30)
plt.xlabel(r'$\Delta r$')
plt.ylabel(r'$N$')

In [ ]:
start = traj[0].particles.position[test_id]
box = traj[0].configuration.box # this box is constant throughout the sim
out = []
for snap in traj:
    next_pos = snap.particles.position[test_id]
    wrapped_vec = cube_pbc_wrap(start, next_pos, box)
    norm = la.norm(wrapped_vec)
    out.append(norm)
plt.plot(out)
plt.ylabel(r'$\Delta r$')
plt.xlabel(r'$t$')

In [ ]:
test_id = 200
start = traj[0].particles.position[test_id]
box = traj[0].configuration.box # this box is constant throughout the sim

idx = np.min(np.where(phop[:,test_id] > 0.2))

out = []
for snap in traj[:idx]:
    next_pos = snap.particles.position[test_id]
    wrapped_vec = cube_pbc_wrap(start, next_pos, box)
    norm = la.norm(wrapped_vec)
    out.append(norm)
plt.hist(out, bins=30)
plt.xlabel(r'$\Delta r$')
plt.ylabel(r'$N$')
print(np.percentile(out, 90), idx)

In [ ]:
phop

In [ ]:
perc = []
box = traj[0].configuration.box # this box is constant throughout the sim

max_idx = len(traj)
for test_id in softness_df.ids:
    start = traj[0].particles.position[test_id]
    out = []
    cond = np.where(phop[:,test_id] > 0.2)
    idx = np.min(cond) if len(cond[0]) > 0 else max_idx
    if idx > 0:
        for snap in traj[:idx]:
            next_pos = snap.particles.position[test_id]
            wrapped_vec = cube_pbc_wrap(start, next_pos, box)
            norm = la.norm(wrapped_vec)
            out.append(norm)
        dat = np.percentile(out, 90)
        perc.append(dat)
plt.hist(perc, bins=30)
plt.ylabel(r"$N$")
plt.xlabel(r"$\Delta r$ P90 $t<t(p_{hop}>0.2)$")

In [ ]:
perc = []
box = traj[0].configuration.box # this box is constant throughout the sim

max_idx = len(traj)
for test_id in softness_df.ids:
    start = traj[0].particles.position[test_id]
    out = []
    cond = np.where(phop[:,test_id] > 0.1)
    idx = np.min(cond) if len(cond[0]) > 0 else max_idx
    if idx > 0:
        for snap in traj[:idx]:
            next_pos = snap.particles.position[test_id]
            wrapped_vec = cube_pbc_wrap(start, next_pos, box)
            norm = la.norm(wrapped_vec)
            out.append(norm)
        dat = np.percentile(out, 90)
        perc.append(dat)
plt.hist(perc, bins=30)
plt.ylabel(r"$N$")
plt.xlabel(r"$\Delta r$ P90 $t<t(p_{hop}>0.1)$")

In [ ]:
sf_config = softness.StructureFunctionConfig(0.1, 2.5, 0.1, 4)
softness_df = softness.calc_structure_functions_dataframe_rust(traj[:1], sf_config=sf_config)
softness_df = softness_df[softness_df.labels == 0]
with open("../../../pipeline.pkl", "rb") as f:
    pipe_dict = pickle.load(f)

In [ ]:
def partial_softness_calculation(Xs, pipe):
    shape = Xs.shape
    data = Xs
    pipe_size = pipe[0].n_features_in_
    scaled_data = pipe[0].transform(np.pad(data, ((0,0),(0,pipe_size-shape[1]))))

    out = np.zeros(shape[0])
    for i in range(shape[0]):
        out[i] = np.sum(scaled_data[i,:shape[1]]*pipe[1].coef_[0][:shape[1]]) + pipe[1].intercept_
    return out

In [ ]:
pipeA = pipe_dict["pipe"]
softness_df["softness"] = partial_softness_calculation(np.stack(softness_df.Xs.values), pipeA)

In [ ]:
softness_df.softness.hist()

In [ ]:
bins = np.linspace(-4, 4, 9)
softness_df["cuts"] = pd.cut(softness_df.softness, bins)

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
perc = []
box = traj[0].configuration.box # this box is constant throughout the sim

normal = colors.Normalize(vmin=-4, vmax=4)
cmap = cm.jet

sm = cm.ScalarMappable(cmap=cmap, norm=normal)

max_idx = len(traj)
for group in softness_df.groupby("cuts"):
    for (jdx, row) in group[1].iterrows():
        # break
        test_id = row["ids"]
        start = traj[0].particles.position[test_id]
        out = []
        cond = np.where(phop[:,test_id] > 0.2)
        idx = np.min(cond) if len(cond[0]) > 0 else max_idx
        if idx > 0:
            for snap in traj[:idx]:
                next_pos = snap.particles.position[test_id]
                wrapped_vec = cube_pbc_wrap(start, next_pos, box)
                norm = la.norm(wrapped_vec)
                out.append(norm)
            dat = np.percentile(out, 90)
            perc.append(dat)
    plt.hist(perc, bins=30, color=cmap(normal(group[0].mid)), density=True, alpha=.5)
    # break
plt.ylabel(r"$N$")
plt.xlabel(r"$\Delta r$ P90 $t<t(p_{hop}>0.2)$")
plt.colorbar(sm, label="S")

In [ ]:
perc = []
box = traj[0].configuration.box # this box is constant throughout the sim

normal = colors.Normalize(vmin=-4, vmax=4)
cmap = cm.jet

sm = cm.ScalarMappable(cmap=cmap, norm=normal)

max_idx = len(traj)
for group in softness_df.groupby("cuts"):
    for (jdx, row) in group[1].iterrows():
        # break
        test_id = row["ids"]
        start = traj[0].particles.position[test_id]
        out = []
        cond = np.where(phop[:,test_id] > 0.1)
        idx = np.min(cond) if len(cond[0]) > 0 else max_idx
        if idx > 0:
            for snap in traj[:idx]:
                next_pos = snap.particles.position[test_id]
                wrapped_vec = cube_pbc_wrap(start, next_pos, box)
                norm = la.norm(wrapped_vec)
                out.append(norm)
            dat = np.percentile(out, 90)
            perc.append(dat)
    plt.hist(perc, bins=30, color=cmap(normal(group[0].mid)), density=True, alpha=.5)
    # break
plt.ylabel(r"$N$")
plt.xlabel(r"$\Delta r$ P90 $t<t(p_{hop}>0.1)$")
plt.colorbar(sm, label="S")